In [1]:
import numpy as np
import pandas as pd
from itertools import permutations, combinations_with_replacement

In [2]:
import spacy

In [3]:
from experiments import CHAR_VOCAB_PATH
from utils import generate_vocab_mappings, load_sentences

In [4]:
itos, stoi = generate_vocab_mappings(CHAR_VOCAB_PATH)

In [5]:
sentences_w_number = load_sentences("input_sentences/refactored_sentences.txt")

In [6]:
nlp = spacy.load('de_core_news_sm')

### Generating german dictionary 

In [7]:
def generate_german_dict(path):
    # TODO: path is "vocabularies/de_dict.csv"
    df = pd.read_csv(path)
    df = df[['lemma','genus','akkusativ singular','akkusativ singular 1','dativ singular',
                'dativ singular 1','genitiv singular','genitiv singular 1',
             'nominativ plural','akkusativ plural','akkusativ plural 1','dativ plural',
                'dativ plural 1','genitiv plural','genitiv plural 1']]
    df = df.rename(columns={
                                 'lemma':'nom sg',
                                 'akkusativ singular':'acc sg',
                                 'akkusativ singular 1':'acc1 sg',
                                 'dativ singular':'dat sg',
                                 'dativ singular 1':'dat1 sg',
                                 'genitiv singular':'gen sg',
                                 'genitiv singular 1':'gen1 sg',
                                 'nominativ plural':'nom pl',
                                 'akkusativ plural':'acc pl',
                                 'akkusativ plural 1':'acc1 pl',
                                 'dativ plural':'dat pl',
                                 'dativ plural 1':'dat1 pl',
                                 'genitiv plural':'gen pl',
                                 'genitiv plural 1':'gen1 pl'
                                 })
    
    df = df.replace(np.nan, '', regex=True)
    df['acc sg'] = df['acc sg'] + df['acc1 sg']
    df['dat sg'] = df['dat sg'] + df['dat1 sg']
    df['gen sg'] = df['gen sg'] + df['gen1 sg']
    df['acc pl'] = df['acc pl'] + df['acc1 pl']
    df['dat pl'] = df['dat pl'] + df['dat1 pl']
    df['gen pl'] = df['gen pl'] + df['gen1 pl']
    df = df.drop(['acc1 sg','dat1 sg','gen1 sg','acc1 pl','dat1 pl','gen1 pl'], axis=1)
    #duplicates = df.duplicated(subset=None, keep='first')
    print(df.shape)
    german_dict = df.set_index('nom sg').T.to_dict('list')
    return german_dict, df

In [8]:
german_dict, df = generate_german_dict("vocabularies/de_dict.csv")

/Users/charlotterochereau/anaconda3/envs/char_lm_fb/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3209: DtypeWarning: Columns (5,9,18,27,36,45,54,63,72) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


(79203, 9)


/Users/charlotterochereau/anaconda3/envs/char_lm_fb/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: DataFrame columns are not unique, some columns will be omitted.


In [9]:
# testing german_dict
print(german_dict["Geschenk"])
print(german_dict["Mann"])
print(german_dict["Junge"])
print(german_dict["Journalist"])
print(german_dict["Präsident"])
print(german_dict["Mädchen"])
print(german_dict["Bett"])
print(german_dict["Held"])
print(german_dict["Arzt"])
print(german_dict["Tisch"])
print(german_dict["Platz"])

['n', 'Geschenk', 'Geschenk', 'Geschenkes', 'Geschenke', 'Geschenke', 'Geschenken', 'Geschenke']
['m', 'Mann', 'Mann', 'Mannes', '', 'Männer', 'Männern', 'Männer']
['m', 'Jungen', 'Jungen', 'Jungen', '', 'Jungen', 'Jungen', 'Jungen']
['m', 'Journalisten', 'Journalisten', 'Journalisten', 'Journalisten', 'Journalisten', 'Journalisten', 'Journalisten']
['m', 'Präsidenten', 'Präsidenten', 'Präsidenten', 'Präsidenten', 'Präsidenten', 'Präsidenten', 'Präsidenten']
['n', 'Mädchen', 'Mädchen', 'Mädchens', 'Mädchen', 'Mädchen', 'Mädchen', 'Mädchen']
['n', 'Bett', 'Bett', 'Betts', 'Betten', 'Betten', 'Betten', 'Betten']
['m', 'Helden', 'Helden', 'Helden', '', 'Helden', 'Helden', 'Helden']
['m', 'Arzt', 'Arzt', 'Arztes', 'Ärzte', 'Ärzte', 'Ärzten', 'Ärzte']
['m', 'Tisch', 'Tisch', 'Tischs', 'Tische', 'Tische', 'Tischen', 'Tische']
['m', 'Platz', 'Platz', 'Platzes', 'Plätze', 'Plätze', 'Plätzen', 'Plätze']


### Parsing using spacy

In [10]:
print(sentences_w_number)

Wir glauben, dass der Mann sg die Frau pl ein Geschenk sg gegeben hat. Wir wissen, dass der Bankier sg sein Kunde pl eine Finanzierung sg angeboten hat. Wir wissen, dass das Kindermädchen sg das Kind sg ein Buch sg gekauft hat. Wir vermuten, dass die Hausherrin sg sein Gast pl ein Wein sg angebietet hat. Wir denken, dass der Mann sg sein Freund pl eine Sehenswürdigkeit pl gezeigt hat. Wir bezweifeln, dass der Verdächtiger sg sein Anwalt sg die Wahrheit sg verheimlicht hat. Wir denken, dass der Arzt sg der Patient sg eine Behandlung sg verschieben hat. Wir erfahren, dass der Eigentümer sg ein Student sg seine Wohnung sg vermietet hat. Wir verdächtigen, dass der Spion sg ihr Feind sg das Geheimnis pl verratet hat. Wir denken, dass die Frau sg sein Partner sg eine Überraschung sg versprochen hat. Ihr weisst, dass die Mutter sg ihre Tochter sg ein Paket sg geschickt hat. Sie sagen, dass der Sportler sg seine Nachbarin sg sein Fahrrad sg verliehen hat. Wir erfahren, dass die Presse sg der Z

In [11]:
sentences_wo_number = sentences_w_number.replace('sg ','').replace('pl ','')

In [12]:
print(sentences_wo_number)

Wir glauben, dass der Mann die Frau ein Geschenk gegeben hat. Wir wissen, dass der Bankier sein Kunde eine Finanzierung angeboten hat. Wir wissen, dass das Kindermädchen das Kind ein Buch gekauft hat. Wir vermuten, dass die Hausherrin sein Gast ein Wein angebietet hat. Wir denken, dass der Mann sein Freund eine Sehenswürdigkeit gezeigt hat. Wir bezweifeln, dass der Verdächtiger sein Anwalt die Wahrheit verheimlicht hat. Wir denken, dass der Arzt der Patient eine Behandlung verschieben hat. Wir erfahren, dass der Eigentümer ein Student seine Wohnung vermietet hat. Wir verdächtigen, dass der Spion ihr Feind das Geheimnis verratet hat. Wir denken, dass die Frau sein Partner eine Überraschung versprochen hat. Ihr weisst, dass die Mutter ihre Tochter ein Paket geschickt hat. Sie sagen, dass der Sportler seine Nachbarin sein Fahrrad verliehen hat. Wir erfahren, dass die Presse der Zuhörer die Nachricht verkündet hat. Sie hören, dass der Senat die Abgeordnete eine Maßnahme gemeldet hat. Ich g

In [13]:
doc_w_number = nlp(sentences_w_number)
print("doc with number:",len(doc_w_number))
doc_wo_number = nlp(sentences_wo_number)
print("doc without number:",len(doc_wo_number))

doc with number: 304
doc without number: 247


### Sentence segmenter

In [14]:
list_sentences_w_number = []
for sentence in doc_w_number.sents:
    list_sentences_w_number.append(sentence.text)
print(list_sentences_w_number)

['Wir glauben, dass der Mann sg die Frau pl ein Geschenk sg gegeben hat.', 'Wir wissen, dass der Bankier sg sein Kunde pl eine Finanzierung sg angeboten hat.', 'Wir wissen, dass das Kindermädchen sg das Kind sg ein Buch sg gekauft hat.', 'Wir vermuten, dass die Hausherrin sg sein Gast pl ein Wein sg angebietet hat.', 'Wir denken, dass der Mann sg sein Freund pl eine Sehenswürdigkeit pl gezeigt hat.', 'Wir bezweifeln, dass der Verdächtiger sg sein Anwalt sg die Wahrheit sg verheimlicht hat.', 'Wir denken, dass der Arzt sg der Patient sg eine Behandlung sg verschieben hat.', 'Wir erfahren, dass der Eigentümer sg ein Student sg seine Wohnung sg vermietet hat.', 'Wir verdächtigen, dass der Spion sg ihr Feind sg das Geheimnis pl verratet hat.', 'Wir denken, dass die Frau sg sein Partner sg eine Überraschung sg versprochen hat.', 'Ihr weisst, dass die Mutter sg ihre Tochter sg ein Paket sg geschickt hat.', 'Sie sagen, dass der Sportler sg seine Nachbarin sg sein Fahrrad sg verliehen hat.', '

In [15]:
list_sentences_wo_number = []
for sentence in doc_wo_number.sents:
    list_sentences_wo_number.append(sentence.text)
print(list_sentences_wo_number)

['Wir glauben, dass der Mann die Frau ein Geschenk gegeben hat.', 'Wir wissen, dass der Bankier sein Kunde eine Finanzierung angeboten hat.', 'Wir wissen, dass das Kindermädchen das Kind ein Buch gekauft hat.', 'Wir vermuten, dass die Hausherrin sein Gast ein Wein angebietet hat.', 'Wir denken, dass der Mann sein Freund eine Sehenswürdigkeit gezeigt hat.', 'Wir bezweifeln, dass der Verdächtiger sein Anwalt die Wahrheit verheimlicht hat.', 'Wir denken, dass der Arzt der Patient eine Behandlung verschieben hat.', 'Wir erfahren, dass der Eigentümer ein Student seine Wohnung vermietet hat.', 'Wir verdächtigen, dass der Spion ihr Feind das Geheimnis verratet hat.', 'Wir denken, dass die Frau sein Partner eine Überraschung versprochen hat.', 'Ihr weisst, dass die Mutter ihre Tochter ein Paket geschickt hat.', 'Sie sagen, dass der Sportler seine Nachbarin sein Fahrrad verliehen hat.', 'Wir erfahren, dass die Presse der Zuhörer die Nachricht verkündet hat.', 'Sie hören, dass der Senat die Abge

In [16]:
assert len(list_sentences_w_number) == len(list_sentences_wo_number)
print("number of sentences:",len(list_sentences_w_number))
print("characters in sentences with number:", len(list_sentences_w_number[0]))
print("characters in sentences without number:", len(list_sentences_wo_number[0]))

number of sentences: 19
characters in sentences with number: 70
characters in sentences without number: 61


In [17]:
number_sentences = len(list_sentences_wo_number)
print(number_sentences)

19


### Tokenizing using spacy

In [18]:
doc_list_w_number = []
for sentence in range(number_sentences):
    doc_list_w_number.append(nlp(list_sentences_w_number[sentence]))

In [19]:
print(doc_list_w_number)

[Wir glauben, dass der Mann sg die Frau pl ein Geschenk sg gegeben hat., Wir wissen, dass der Bankier sg sein Kunde pl eine Finanzierung sg angeboten hat., Wir wissen, dass das Kindermädchen sg das Kind sg ein Buch sg gekauft hat., Wir vermuten, dass die Hausherrin sg sein Gast pl ein Wein sg angebietet hat., Wir denken, dass der Mann sg sein Freund pl eine Sehenswürdigkeit pl gezeigt hat., Wir bezweifeln, dass der Verdächtiger sg sein Anwalt sg die Wahrheit sg verheimlicht hat., Wir denken, dass der Arzt sg der Patient sg eine Behandlung sg verschieben hat., Wir erfahren, dass der Eigentümer sg ein Student sg seine Wohnung sg vermietet hat., Wir verdächtigen, dass der Spion sg ihr Feind sg das Geheimnis pl verratet hat., Wir denken, dass die Frau sg sein Partner sg eine Überraschung sg versprochen hat., Ihr weisst, dass die Mutter sg ihre Tochter sg ein Paket sg geschickt hat., Sie sagen, dass der Sportler sg seine Nachbarin sg sein Fahrrad sg verliehen hat., Wir erfahren, dass die Pr

In [20]:
tokens_w_number = [[] for _ in range(number_sentences)]
for idx_sentence, sentence in enumerate(doc_list_w_number):
    for token in sentence: 
        tokens_w_number[idx_sentence].append(token.text)

In [21]:
doc_list_wo_number = []
for sentence in range(number_sentences):
    doc_list_wo_number.append(nlp(list_sentences_wo_number[sentence]))

In [22]:
print(doc_list_wo_number)

[Wir glauben, dass der Mann die Frau ein Geschenk gegeben hat., Wir wissen, dass der Bankier sein Kunde eine Finanzierung angeboten hat., Wir wissen, dass das Kindermädchen das Kind ein Buch gekauft hat., Wir vermuten, dass die Hausherrin sein Gast ein Wein angebietet hat., Wir denken, dass der Mann sein Freund eine Sehenswürdigkeit gezeigt hat., Wir bezweifeln, dass der Verdächtiger sein Anwalt die Wahrheit verheimlicht hat., Wir denken, dass der Arzt der Patient eine Behandlung verschieben hat., Wir erfahren, dass der Eigentümer ein Student seine Wohnung vermietet hat., Wir verdächtigen, dass der Spion ihr Feind das Geheimnis verratet hat., Wir denken, dass die Frau sein Partner eine Überraschung versprochen hat., Ihr weisst, dass die Mutter ihre Tochter ein Paket geschickt hat., Sie sagen, dass der Sportler seine Nachbarin sein Fahrrad verliehen hat., Wir erfahren, dass die Presse der Zuhörer die Nachricht verkündet hat., Sie hören, dass der Senat die Abgeordnete eine Maßnahme gemel

In [23]:
tokens_wo_number = [[] for _ in range(number_sentences)]
for idx_sentence, sentence in enumerate(doc_list_wo_number):
    for token in sentence: 
        tokens_wo_number[idx_sentence].append(token.text)

In [24]:
print(tokens_wo_number)

[['Wir', 'glauben', ',', 'dass', 'der', 'Mann', 'die', 'Frau', 'ein', 'Geschenk', 'gegeben', 'hat', '.'], ['Wir', 'wissen', ',', 'dass', 'der', 'Bankier', 'sein', 'Kunde', 'eine', 'Finanzierung', 'angeboten', 'hat', '.'], ['Wir', 'wissen', ',', 'dass', 'das', 'Kindermädchen', 'das', 'Kind', 'ein', 'Buch', 'gekauft', 'hat', '.'], ['Wir', 'vermuten', ',', 'dass', 'die', 'Hausherrin', 'sein', 'Gast', 'ein', 'Wein', 'angebietet', 'hat', '.'], ['Wir', 'denken', ',', 'dass', 'der', 'Mann', 'sein', 'Freund', 'eine', 'Sehenswürdigkeit', 'gezeigt', 'hat', '.'], ['Wir', 'bezweifeln', ',', 'dass', 'der', 'Verdächtiger', 'sein', 'Anwalt', 'die', 'Wahrheit', 'verheimlicht', 'hat', '.'], ['Wir', 'denken', ',', 'dass', 'der', 'Arzt', 'der', 'Patient', 'eine', 'Behandlung', 'verschieben', 'hat', '.'], ['Wir', 'erfahren', ',', 'dass', 'der', 'Eigentümer', 'ein', 'Student', 'seine', 'Wohnung', 'vermietet', 'hat', '.'], ['Wir', 'verdächtigen', ',', 'dass', 'der', 'Spion', 'ihr', 'Feind', 'das', 'Geheimni

In [25]:
print(tokens_wo_number)

[['Wir', 'glauben', ',', 'dass', 'der', 'Mann', 'die', 'Frau', 'ein', 'Geschenk', 'gegeben', 'hat', '.'], ['Wir', 'wissen', ',', 'dass', 'der', 'Bankier', 'sein', 'Kunde', 'eine', 'Finanzierung', 'angeboten', 'hat', '.'], ['Wir', 'wissen', ',', 'dass', 'das', 'Kindermädchen', 'das', 'Kind', 'ein', 'Buch', 'gekauft', 'hat', '.'], ['Wir', 'vermuten', ',', 'dass', 'die', 'Hausherrin', 'sein', 'Gast', 'ein', 'Wein', 'angebietet', 'hat', '.'], ['Wir', 'denken', ',', 'dass', 'der', 'Mann', 'sein', 'Freund', 'eine', 'Sehenswürdigkeit', 'gezeigt', 'hat', '.'], ['Wir', 'bezweifeln', ',', 'dass', 'der', 'Verdächtiger', 'sein', 'Anwalt', 'die', 'Wahrheit', 'verheimlicht', 'hat', '.'], ['Wir', 'denken', ',', 'dass', 'der', 'Arzt', 'der', 'Patient', 'eine', 'Behandlung', 'verschieben', 'hat', '.'], ['Wir', 'erfahren', ',', 'dass', 'der', 'Eigentümer', 'ein', 'Student', 'seine', 'Wohnung', 'vermietet', 'hat', '.'], ['Wir', 'verdächtigen', ',', 'dass', 'der', 'Spion', 'ihr', 'Feind', 'das', 'Geheimni

In [26]:
print("sentences in tokens", len(tokens_w_number))
print("characters in tokens with number:", len(tokens_w_number[0]))
print("characters in tokens without number:", len(tokens_wo_number[0]))

sentences in tokens 19
characters in tokens with number: 16
characters in tokens without number: 13


### Selecting verb arguments

In [ ]:
# Backup
def select_args_wo_nb(args_nb, nb_sentences, tokens):

    verb_args = [[] for _ in range(nb_sentences)]
    for sentence_idx, sentence in enumerate(tokens):
        verb_args[sentence_idx].append(tokens[sentence_idx][4] + " " + tokens[sentence_idx][5])
        verb_args[sentence_idx].append(tokens[sentence_idx][6] + " " + tokens[sentence_idx][7])
        verb_args[sentence_idx].append(tokens[sentence_idx][8] + " " + tokens[sentence_idx][9])

    return verb_args

In [27]:
args_wo_number = [[] for _ in range(number_sentences)]
for sentence_idx, sentence in enumerate(tokens_wo_number):
    args_wo_number[sentence_idx].append(tokens_wo_number[sentence_idx][4]+" "+tokens_wo_number[sentence_idx][5])
    args_wo_number[sentence_idx].append(tokens_wo_number[sentence_idx][6]+" "+tokens_wo_number[sentence_idx][7])
    args_wo_number[sentence_idx].append(tokens_wo_number[sentence_idx][8]+" "+tokens_wo_number[sentence_idx][9])
print(args_wo_number)

[['der Mann', 'die Frau', 'ein Geschenk'], ['der Bankier', 'sein Kunde', 'eine Finanzierung'], ['das Kindermädchen', 'das Kind', 'ein Buch'], ['die Hausherrin', 'sein Gast', 'ein Wein'], ['der Mann', 'sein Freund', 'eine Sehenswürdigkeit'], ['der Verdächtiger', 'sein Anwalt', 'die Wahrheit'], ['der Arzt', 'der Patient', 'eine Behandlung'], ['der Eigentümer', 'ein Student', 'seine Wohnung'], ['der Spion', 'ihr Feind', 'das Geheimnis'], ['die Frau', 'sein Partner', 'eine Überraschung'], ['die Mutter', 'ihre Tochter', 'ein Paket'], ['der Sportler', 'seine Nachbarin', 'sein Fahrrad'], ['die Presse', 'der Zuhörer', 'die Nachricht'], ['der Senat', 'die Abgeordnete', 'eine Maßnahme'], ['der Junge', 'sein Kumpel', 'ein Kugelschreiber'], ['das Theater', 'der Musiker', 'ein Sitzplatz'], ['der Schüler', 'der Lehrer', 'ihre Frage'], ['der Professor', 'der Student', 'ein Fach'], ['der Chef', 'das Kind', 'ein Kuchen']]


In [28]:
args_number = 3
args_w_number = [[[] for _ in range(args_number)] for _ in range(number_sentences)]
for sentence_idx, sentence in enumerate(tokens_w_number):
    args_w_number[sentence_idx][0]=(tokens_w_number[sentence_idx][4:7])
    args_w_number[sentence_idx][1]=(tokens_w_number[sentence_idx][7:10])
    args_w_number[sentence_idx][2]=(tokens_w_number[sentence_idx][10:13])
print(args_w_number)

[[['der', 'Mann', 'sg'], ['die', 'Frau', 'pl'], ['ein', 'Geschenk', 'sg']], [['der', 'Bankier', 'sg'], ['sein', 'Kunde', 'pl'], ['eine', 'Finanzierung', 'sg']], [['das', 'Kindermädchen', 'sg'], ['das', 'Kind', 'sg'], ['ein', 'Buch', 'sg']], [['die', 'Hausherrin', 'sg'], ['sein', 'Gast', 'pl'], ['ein', 'Wein', 'sg']], [['der', 'Mann', 'sg'], ['sein', 'Freund', 'pl'], ['eine', 'Sehenswürdigkeit', 'pl']], [['der', 'Verdächtiger', 'sg'], ['sein', 'Anwalt', 'sg'], ['die', 'Wahrheit', 'sg']], [['der', 'Arzt', 'sg'], ['der', 'Patient', 'sg'], ['eine', 'Behandlung', 'sg']], [['der', 'Eigentümer', 'sg'], ['ein', 'Student', 'sg'], ['seine', 'Wohnung', 'sg']], [['der', 'Spion', 'sg'], ['ihr', 'Feind', 'sg'], ['das', 'Geheimnis', 'pl']], [['die', 'Frau', 'sg'], ['sein', 'Partner', 'sg'], ['eine', 'Überraschung', 'sg']], [['die', 'Mutter', 'sg'], ['ihre', 'Tochter', 'sg'], ['ein', 'Paket', 'sg']], [['der', 'Sportler', 'sg'], ['seine', 'Nachbarin', 'sg'], ['sein', 'Fahrrad', 'sg']], [['die', 'Presse

In [29]:
print(len(args_w_number))
print(len(args_w_number[0]))

19
3


### Generate case permutations

In [30]:
def get_nom(group_args):
    # get nominative of verb argument and corresponding article
    new_nom = []
    article = group_args[0]
    word = group_args[1]
    number = group_args[2]
    
    if number == "sg":
        new_nom.append(article+" "+word)

    elif number == "pl":
        pl_article = ""
        pl_word = ""
        if article in {"der","die","das"}:
            pl_article = "die"
        elif article in {"sein","seine"}:
            pl_article = "seine"
        elif article in {"ihr","ihre"}:
            pl_article = "ihre"
        elif article in {"ein","eine"}:
            pl_article = "einige"
        
        pl_word = german_dict[word][4]
        new_nom.append(pl_article+" "+pl_word)
        
    return new_nom

In [31]:
print(get_nom(args_w_number[0][0]))
print(get_nom(args_w_number[0][1]))
print(get_nom(args_w_number[17][2]))
print(get_nom(args_w_number[18][2]))

['der Mann']
['die Frauen']
['einige Fächer']
['ein Kuchen']


In [32]:
def get_acc(group_args):
    # get accusative of verb argument and corresponding article
    new_acc = []
    article = group_args[0]
    word = group_args[1]
    gender = german_dict[word][0]
    number = group_args[2]

    if number == "sg":
        sg_article = ""
        sg_word = ""
        if gender == "m":
            if article == "der":
                sg_article = "den"
            elif article == "sein":
                sg_article = "seinen"
            elif article == "ihr":
                sg_article = "ihren"
            else:
                sg_article = "einen"
        else:
            sg_article = article
        
        sg_word = german_dict[word][1]
        
        new_acc.append(sg_article+" "+sg_word)
        
    elif number == "pl":
        pl_article = ""
        pl_word = ""
        if article in {"der","die","das"}:
            pl_article = "die"
        elif article in {"sein","seine"}:
            pl_article = "seine"
        elif article in {"ihr","ihre"}:
            pl_article = "ihre"
        elif article in {"ein","eine"}:
            pl_article = "einige"
            
        pl_word = german_dict[word][5]   
        new_acc.append(pl_article+" "+pl_word)
        
    return new_acc

In [33]:
print(get_acc(args_w_number[0][0]))
print(get_acc(args_w_number[0][1]))
print(get_acc(args_w_number[1][0]))
print(get_acc(args_w_number[17][2]))

['den Mann']
['die Frauen']
['den Bankier']
['einige Fächer']


In [34]:
def get_dat(group_args):
    # get dative of verb argument and corresponding article
    new_dat = []
    article = group_args[0]
    word = group_args[1]
    gender = german_dict[word][0]
    number = group_args[2]
    
    if number == "sg":
        sg_article = ""
        sg_word = ""
        if gender in {"m","n"}:
            if article in {"der","das"}:
                sg_article = "dem"
            elif article == "sein":
                sg_article = "seinem"
            elif article == "ihr":
                sg_article = "ihrem"
            elif article == "ein":
                sg_article = "einem"
        elif gender == "f":
            if article == "die":
                sg_article = "der"
            elif article == "seine":
                sg_article = "seiner"
            elif article == "ihre":
                sg_article = "ihrer"
            elif article == "eine":
                sg_article = "einer"
        
        sg_word = german_dict[word][2] 
        new_dat.append(sg_article+" "+sg_word)
        
    elif number == "pl": 
        pl_article = ""
        pl_word = ""
        if article in {"der","die","das"}:
            pl_article = "den"
        elif article in {"sein","seine"}:
            pl_article = "seinen"
        elif article in {"ihr","ihre"}:
            pl_article = "ihren" 
        elif article in {"ein","eine"}:
            pl_article = "einigen"
    
        pl_word = german_dict[word][6]     
        new_dat.append(pl_article+" "+pl_word)
        
    return new_dat

In [35]:
print(get_dat(args_w_number[0][0]))
print(get_dat(args_w_number[0][1]))
print(get_dat(args_w_number[17][0]))
print(get_dat(args_w_number[17][1]))
print(get_dat(args_w_number[17][2]))
print(get_nom(args_w_number[18][2]))

['dem Mann']
['den Frauen']
['dem Professor']
['den Studenten']
['einigen Fächern']
['ein Kuchen']


In [36]:
def get_all_cases(list_args, number_args):
    all_cases = [[[] for _ in range(number_args)] for _ in range(number_sentences)]
    for sentence_idx, sentence in enumerate(args_w_number):
        for group_args_idx, group_args in enumerate(args_w_number[sentence_idx]):
            nom = get_nom(args_w_number[sentence_idx][group_args_idx])
            acc = get_acc(args_w_number[sentence_idx][group_args_idx])
            dat = get_dat(args_w_number[sentence_idx][group_args_idx])
            all_cases[sentence_idx][group_args_idx].append(nom)
            all_cases[sentence_idx][group_args_idx].append(acc)
            all_cases[sentence_idx][group_args_idx].append(dat)
    return all_cases

In [79]:
case_permutations = get_all_cases(args_w_number, 3)
print(len(case_permutations))
print(len(case_permutations[0]))
print(case_permutations)

19
3
[[[['der Mann'], ['den Mann'], ['dem Mann']], [['die Frauen'], ['die Frauen'], ['den Frauen']], [['ein Geschenk'], ['ein Geschenk'], ['einem Geschenk']]], [[['der Bankier'], ['den Bankier'], ['dem Bankier']], [['seine Kunden'], ['seine Kunden'], ['seinen Kunden']], [['eine Finanzierung'], ['eine Finanzierung'], ['einer Finanzierung']]], [[['das Kindermädchen'], ['das Kindermädchen'], ['dem Kindermädchen']], [['das Kind'], ['das Kind'], ['dem Kind']], [['ein Buch'], ['ein Buch'], ['einem Buch']]], [[['die Hausherrin'], ['die Hausherrin'], ['der Hausherrin']], [['seine Gasten'], ['seine Gasten'], ['seinen Gasten']], [['ein Wein'], ['einen Wein'], ['einem Wein']]], [[['der Mann'], ['den Mann'], ['dem Mann']], [['seine Freunde'], ['seine Freunde'], ['seinen Freunden']], [['einige Sehenswürdigkeiten'], ['einige Sehenswürdigkeiten'], ['einigen Sehenswürdigkeiten']]], [[['der Verdächtiger'], ['den Verdächtigen'], ['dem Verdächtigem']], [['sein Anwalt'], ['seinen Anwalt'], ['seinem Anwalt

In [38]:
# 0 = Nom, 1 = Acc, 2 = Dat
idxs = [0,1,2]
perm = list(permutations(idxs))
print(perm)

[(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)]


In [39]:
combinations = [[[] for _ in range(6)] for _ in range(number_sentences)]
for stc_idx, stc in enumerate(case_permutations):
    for idx in range(6):
        for sub_idx in range(3):
            perm_idx = perm[idx][sub_idx]
            combinations[stc_idx][idx].append(case_permutations[stc_idx][sub_idx][perm_idx])

In [40]:
print(len(combinations))
print(len(combinations[0]))
print(combinations[0])

19
6
[[['der Mann'], ['die Frauen'], ['einem Geschenk']], [['der Mann'], ['den Frauen'], ['ein Geschenk']], [['den Mann'], ['die Frauen'], ['einem Geschenk']], [['den Mann'], ['den Frauen'], ['ein Geschenk']], [['dem Mann'], ['die Frauen'], ['ein Geschenk']], [['dem Mann'], ['die Frauen'], ['ein Geschenk']]]


In [41]:
print(combinations)

[[[['der Mann'], ['die Frauen'], ['einem Geschenk']], [['der Mann'], ['den Frauen'], ['ein Geschenk']], [['den Mann'], ['die Frauen'], ['einem Geschenk']], [['den Mann'], ['den Frauen'], ['ein Geschenk']], [['dem Mann'], ['die Frauen'], ['ein Geschenk']], [['dem Mann'], ['die Frauen'], ['ein Geschenk']]], [[['der Bankier'], ['seine Kunden'], ['einer Finanzierung']], [['der Bankier'], ['seinen Kunden'], ['eine Finanzierung']], [['den Bankier'], ['seine Kunden'], ['einer Finanzierung']], [['den Bankier'], ['seinen Kunden'], ['eine Finanzierung']], [['dem Bankier'], ['seine Kunden'], ['eine Finanzierung']], [['dem Bankier'], ['seine Kunden'], ['eine Finanzierung']]], [[['das Kindermädchen'], ['das Kind'], ['einem Buch']], [['das Kindermädchen'], ['dem Kind'], ['ein Buch']], [['das Kindermädchen'], ['das Kind'], ['einem Buch']], [['das Kindermädchen'], ['dem Kind'], ['ein Buch']], [['dem Kindermädchen'], ['das Kind'], ['ein Buch']], [['dem Kindermädchen'], ['das Kind'], ['ein Buch']]], [[[

In [42]:
all_perm = [[] for _ in range(number_sentences)]
for stc_idx, stc in enumerate(combinations):
    for idx in range(6):
        tmp_perm = list(permutations(combinations[stc_idx][idx]))
        all_perm[stc_idx].append(tmp_perm)

In [43]:
print(all_perm)

[[[(['der Mann'], ['die Frauen'], ['einem Geschenk']), (['der Mann'], ['einem Geschenk'], ['die Frauen']), (['die Frauen'], ['der Mann'], ['einem Geschenk']), (['die Frauen'], ['einem Geschenk'], ['der Mann']), (['einem Geschenk'], ['der Mann'], ['die Frauen']), (['einem Geschenk'], ['die Frauen'], ['der Mann'])], [(['der Mann'], ['den Frauen'], ['ein Geschenk']), (['der Mann'], ['ein Geschenk'], ['den Frauen']), (['den Frauen'], ['der Mann'], ['ein Geschenk']), (['den Frauen'], ['ein Geschenk'], ['der Mann']), (['ein Geschenk'], ['der Mann'], ['den Frauen']), (['ein Geschenk'], ['den Frauen'], ['der Mann'])], [(['den Mann'], ['die Frauen'], ['einem Geschenk']), (['den Mann'], ['einem Geschenk'], ['die Frauen']), (['die Frauen'], ['den Mann'], ['einem Geschenk']), (['die Frauen'], ['einem Geschenk'], ['den Mann']), (['einem Geschenk'], ['den Mann'], ['die Frauen']), (['einem Geschenk'], ['die Frauen'], ['den Mann'])], [(['den Mann'], ['den Frauen'], ['ein Geschenk']), (['den Mann'], ['

In [44]:
print(len(all_perm))
print(len(all_perm[0]))
print(len(all_perm[0][0]))
print(len(all_perm[0][0][0]))

19
6
6
3


In [45]:
#print(all_perm[0])
#print(all_perm[0][0])
print(all_perm[0][0][0])

(['der Mann'], ['die Frauen'], ['einem Geschenk'])


### Generate grammatical sentences

In [49]:
all_gramm = ""
for stc_idx, stc in enumerate(tokens_wo_number): 
    for case_perm_idx, case_perm in enumerate(all_perm[stc_idx]):
        for case_idx, case in enumerate(all_perm[stc_idx][case_perm_idx]):
            principal_clause = f"{tokens_wo_number[stc_idx][0]} {tokens_wo_number[stc_idx][1]}, {tokens_wo_number[stc_idx][3]}"
            
            verb_arg = ""
            for vb_arg_idx, vb_arg in enumerate(all_perm[stc_idx][case_perm_idx][case_idx]):
                verb_arg += f"{vb_arg[0]} " 

            verb_subordinate = f"{tokens_wo_number[stc_idx][-3]} {tokens_wo_number[stc_idx][-2]}"
            all_gramm += (f"{principal_clause} {verb_arg}{verb_subordinate}.\n")
        
print(len(all_gramm))


50136


In [50]:
print(all_gramm)

Wir glauben, dass der Mann die Frauen einem Geschenk gegeben hat.
Wir glauben, dass der Mann einem Geschenk die Frauen gegeben hat.
Wir glauben, dass die Frauen der Mann einem Geschenk gegeben hat.
Wir glauben, dass die Frauen einem Geschenk der Mann gegeben hat.
Wir glauben, dass einem Geschenk der Mann die Frauen gegeben hat.
Wir glauben, dass einem Geschenk die Frauen der Mann gegeben hat.
Wir glauben, dass der Mann den Frauen ein Geschenk gegeben hat.
Wir glauben, dass der Mann ein Geschenk den Frauen gegeben hat.
Wir glauben, dass den Frauen der Mann ein Geschenk gegeben hat.
Wir glauben, dass den Frauen ein Geschenk der Mann gegeben hat.
Wir glauben, dass ein Geschenk der Mann den Frauen gegeben hat.
Wir glauben, dass ein Geschenk den Frauen der Mann gegeben hat.
Wir glauben, dass den Mann die Frauen einem Geschenk gegeben hat.
Wir glauben, dass den Mann einem Geschenk die Frauen gegeben hat.
Wir glauben, dass die Frauen den Mann einem Geschenk gegeben hat.
Wir glauben, dass die 

In [51]:
BASE_PATH = "input_sentences/"
output_file = open(BASE_PATH + 'grammatical_sentences.txt', 'w')
output_file.write(all_gramm)
output_file.close()

### Generate ungrammatical sentences

In [53]:
# 0 = Nom, 1 = Acc, 2 = Dat
nb_cases = [0,1,2]
nb_case_repeats = list(combinations_with_replacement(nb_cases,3))
print(len(nb_case_repeats),[x for x in nb_case_repeats])

10 [(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 1, 1), (0, 1, 2), (0, 2, 2), (1, 1, 1), (1, 1, 2), (1, 2, 2), (2, 2, 2)]


In [54]:
for idx, value in enumerate(nb_case_repeats):
    if value in {(0,1,2),(0,0,0),(1,1,1),(2,2,2)}:
        nb_case_repeats.remove(value)
print(len(nb_case_repeats),nb_case_repeats)

6 [(0, 0, 1), (0, 0, 2), (0, 1, 1), (0, 2, 2), (1, 1, 2), (1, 2, 2)]


In [55]:
idxs_perms = [[] for _ in range(len(nb_case_repeats))]
for idx, value in enumerate(nb_case_repeats):
    tmp_idx_perm = list(permutations(value))
    idxs_perms[idx] = tmp_idx_perm
print(idxs_perms)
print(len(idxs_perms))

[[(0, 0, 1), (0, 1, 0), (0, 0, 1), (0, 1, 0), (1, 0, 0), (1, 0, 0)], [(0, 0, 2), (0, 2, 0), (0, 0, 2), (0, 2, 0), (2, 0, 0), (2, 0, 0)], [(0, 1, 1), (0, 1, 1), (1, 0, 1), (1, 1, 0), (1, 0, 1), (1, 1, 0)], [(0, 2, 2), (0, 2, 2), (2, 0, 2), (2, 2, 0), (2, 0, 2), (2, 2, 0)], [(1, 1, 2), (1, 2, 1), (1, 1, 2), (1, 2, 1), (2, 1, 1), (2, 1, 1)], [(1, 2, 2), (1, 2, 2), (2, 1, 2), (2, 2, 1), (2, 1, 2), (2, 2, 1)]]
6


In [56]:
ungram_idxs = []
for idx, value in enumerate(idxs_perms):
    for i in set(idxs_perms[idx]):
        ungram_idxs.append(i)
print(ungram_idxs)
print(len(ungram_idxs))

[(1, 0, 0), (0, 1, 0), (0, 0, 1), (0, 2, 0), (2, 0, 0), (0, 0, 2), (0, 1, 1), (1, 1, 0), (1, 0, 1), (2, 2, 0), (2, 0, 2), (0, 2, 2), (1, 2, 1), (2, 1, 1), (1, 1, 2), (1, 2, 2), (2, 2, 1), (2, 1, 2)]
18


In [82]:
print(len(case_permutations))
print(len(case_permutations[0]))

19
3


In [83]:
ungram_combinations = [[[]for _ in range(len(ungram_idxs))] for _ in range(number_sentences)]
for stc_idx, stc in enumerate(case_permutations):
    for case_violation_idx, case_violation in enumerate(ungram_idxs):
        for sub_idx in range(3):
            perm_idx = ungram_idxs[case_violation_idx][sub_idx]
            ungram_combinations[stc_idx][case_violation_idx].append(case_permutations[stc_idx][sub_idx][perm_idx])

print('sentences:',len(ungram_combinations))
print('cases combinations:',len(ungram_combinations[0]))
print('permutations of each cases combination:',len(ungram_combinations[0][0]))
print(ungram_combinations[0])

sentences: 19
cases combinations: 18
permutations of each cases combination: 3
[[['den Mann'], ['die Frauen'], ['ein Geschenk']], [['der Mann'], ['die Frauen'], ['ein Geschenk']], [['der Mann'], ['die Frauen'], ['ein Geschenk']], [['der Mann'], ['den Frauen'], ['ein Geschenk']], [['dem Mann'], ['die Frauen'], ['ein Geschenk']], [['der Mann'], ['die Frauen'], ['einem Geschenk']], [['der Mann'], ['die Frauen'], ['ein Geschenk']], [['den Mann'], ['die Frauen'], ['ein Geschenk']], [['den Mann'], ['die Frauen'], ['ein Geschenk']], [['dem Mann'], ['den Frauen'], ['ein Geschenk']], [['dem Mann'], ['die Frauen'], ['einem Geschenk']], [['der Mann'], ['den Frauen'], ['einem Geschenk']], [['den Mann'], ['den Frauen'], ['ein Geschenk']], [['dem Mann'], ['die Frauen'], ['ein Geschenk']], [['den Mann'], ['die Frauen'], ['einem Geschenk']], [['den Mann'], ['den Frauen'], ['einem Geschenk']], [['dem Mann'], ['den Frauen'], ['ein Geschenk']], [['dem Mann'], ['die Frauen'], ['einem Geschenk']]]


In [84]:
all_ungram_perm = [[] for _ in range(number_sentences)]
for stc_idx, stc in enumerate(ungram_combinations):
    for cases_idx, cases in enumerate(ungram_combinations[stc_idx]):
        tmp_perm = list(permutations(ungram_combinations[stc_idx][cases_idx]))
        all_ungram_perm[stc_idx].append(tmp_perm)
print('sentences:',len(all_ungram_perm))
print('cases combinations:',len(all_ungram_perm[0]))
print('permutations of positions:',len(all_ungram_perm[0][0]))
print(all_ungram_perm[0][0])
print(all_ungram_perm[18][1])

sentences: 19
cases combinations: 18
permutations of positions: 6
[(['den Mann'], ['die Frauen'], ['ein Geschenk']), (['den Mann'], ['ein Geschenk'], ['die Frauen']), (['die Frauen'], ['den Mann'], ['ein Geschenk']), (['die Frauen'], ['ein Geschenk'], ['den Mann']), (['ein Geschenk'], ['den Mann'], ['die Frauen']), (['ein Geschenk'], ['die Frauen'], ['den Mann'])]
[(['der Chef'], ['die Kinder'], ['ein Kuchen']), (['der Chef'], ['ein Kuchen'], ['die Kinder']), (['die Kinder'], ['der Chef'], ['ein Kuchen']), (['die Kinder'], ['ein Kuchen'], ['der Chef']), (['ein Kuchen'], ['der Chef'], ['die Kinder']), (['ein Kuchen'], ['die Kinder'], ['der Chef'])]


In [59]:
all_ungramm = ""
counter = 0
for stc_idx, stc in enumerate(tokens_wo_number): 
    for case_perm_idx, case_perm in enumerate(all_ungram_perm[stc_idx]):
        for case_idx, case in enumerate(all_ungram_perm[stc_idx][case_perm_idx]):
            principal_clause = f"{tokens_wo_number[stc_idx][0]} {tokens_wo_number[stc_idx][1]}, {tokens_wo_number[stc_idx][3]}"
            
            verb_arg = ""
            for vb_arg_idx, vb_arg in enumerate(all_ungram_perm[stc_idx][case_perm_idx][case_idx]):
                verb_arg += f"{vb_arg[0]} " 

            verb_subordinate = f"{tokens_wo_number[stc_idx][-3]} {tokens_wo_number[stc_idx][-2]}"
            all_ungramm += (f"{principal_clause} {verb_arg}{verb_subordinate}.\n")            
            counter += 1
            
            
counter = counter/(number_sentences)
        
print(len(all_ungramm))


150408


In [76]:
print(all_ungram_perm[0][0][0][0][0])
for stc_idx, stc in enumerate(tokens_wo_number): 
    for case_perm_idx, case_perm in enumerate(all_ungram_perm[stc_idx]):
        for case_idx, case in enumerate(all_ungram_perm[stc_idx][case_perm_idx]):
            principal_clause = f"{tokens_wo_number[stc_idx][0]} {tokens_wo_number[stc_idx][1]}, {tokens_wo_number[stc_idx][3]}"
            
            verb_arg = ""
            for vb_arg_idx, vb_arg in enumerate(all_ungram_perm[stc_idx][case_perm_idx][case_idx]):
                verb_arg += f"{vb_arg[0]} " 
            print(verb_arg)
                

den Mann
den Mann die Frauen ein Geschenk 
den Mann ein Geschenk die Frauen 
die Frauen den Mann ein Geschenk 
die Frauen ein Geschenk den Mann 
ein Geschenk den Mann die Frauen 
ein Geschenk die Frauen den Mann 
der Mann die Frauen ein Geschenk 
der Mann ein Geschenk die Frauen 
die Frauen der Mann ein Geschenk 
die Frauen ein Geschenk der Mann 
ein Geschenk der Mann die Frauen 
ein Geschenk die Frauen der Mann 
der Mann die Frauen ein Geschenk 
der Mann ein Geschenk die Frauen 
die Frauen der Mann ein Geschenk 
die Frauen ein Geschenk der Mann 
ein Geschenk der Mann die Frauen 
ein Geschenk die Frauen der Mann 
der Mann den Frauen ein Geschenk 
der Mann ein Geschenk den Frauen 
den Frauen der Mann ein Geschenk 
den Frauen ein Geschenk der Mann 
ein Geschenk der Mann den Frauen 
ein Geschenk den Frauen der Mann 
dem Mann die Frauen ein Geschenk 
dem Mann ein Geschenk die Frauen 
die Frauen dem Mann ein Geschenk 
die Frauen ein Geschenk dem Mann 
ein Geschenk dem Mann die Frauen 
ein G

den Lehrer ihre Fragen den Schülern 
ihre Fragen den Schülern den Lehrer 
ihre Fragen den Lehrer den Schülern 
die Schüler den Lehrer ihren Fragen 
die Schüler ihren Fragen den Lehrer 
den Lehrer die Schüler ihren Fragen 
den Lehrer ihren Fragen die Schüler 
ihren Fragen die Schüler den Lehrer 
ihren Fragen den Lehrer die Schüler 
die Schüler dem Lehrer ihren Fragen 
die Schüler ihren Fragen dem Lehrer 
dem Lehrer die Schüler ihren Fragen 
dem Lehrer ihren Fragen die Schüler 
ihren Fragen die Schüler dem Lehrer 
ihren Fragen dem Lehrer die Schüler 
den Schülern dem Lehrer ihre Fragen 
den Schülern ihre Fragen dem Lehrer 
dem Lehrer den Schülern ihre Fragen 
dem Lehrer ihre Fragen den Schülern 
ihre Fragen den Schülern dem Lehrer 
ihre Fragen dem Lehrer den Schülern 
den Schülern den Lehrer ihren Fragen 
den Schülern ihren Fragen den Lehrer 
den Lehrer den Schülern ihren Fragen 
den Lehrer ihren Fragen den Schülern 
ihren Fragen den Schülern den Lehrer 
ihren Fragen den Lehrer den Schül

In [61]:
print(counter)

108.0


In [62]:
print(all_ungramm)

Wir glauben, dass den Mann die Frauen ein Geschenk gegeben hat.
Wir glauben, dass den Mann ein Geschenk die Frauen gegeben hat.
Wir glauben, dass die Frauen den Mann ein Geschenk gegeben hat.
Wir glauben, dass die Frauen ein Geschenk den Mann gegeben hat.
Wir glauben, dass ein Geschenk den Mann die Frauen gegeben hat.
Wir glauben, dass ein Geschenk die Frauen den Mann gegeben hat.
Wir glauben, dass der Mann die Frauen ein Geschenk gegeben hat.
Wir glauben, dass der Mann ein Geschenk die Frauen gegeben hat.
Wir glauben, dass die Frauen der Mann ein Geschenk gegeben hat.
Wir glauben, dass die Frauen ein Geschenk der Mann gegeben hat.
Wir glauben, dass ein Geschenk der Mann die Frauen gegeben hat.
Wir glauben, dass ein Geschenk die Frauen der Mann gegeben hat.
Wir glauben, dass der Mann die Frauen ein Geschenk gegeben hat.
Wir glauben, dass der Mann ein Geschenk die Frauen gegeben hat.
Wir glauben, dass die Frauen der Mann ein Geschenk gegeben hat.
Wir glauben, dass die Frauen ein Geschen

In [ ]:
print('number of sentences:',len(all_ungramm_list))
print('number of ungrammatical sentences per original sentence:',len(all_ungramm_list[0]))

In [77]:
BASE_PATH = "input_sentences/"
output_file = open(BASE_PATH + 'ungrammatical_sentences.txt', 'w')
output_file.write(all_ungramm)
output_file.close()

In [ ]:
def load_sentences(input_path):
    """
    Input: str, path to input file
    Output: loaded file
    """
    with open(input_path, "r") as f:
        output = f.read()

    return output

In [ ]:
with open("input_sentences/ungrammatical_sentences.txt", "r") as f:
     ungrammatical_sentences = f.read().split("\n")

In [ ]:
print(ungrammatical_sentences)

In [ ]:
def load_sentences(input_path):
    """
    Input: str, path to input file
    Output: loaded file
    """
    with open(input_path, "r") as f:
        output = f.read()

    return output

In [ ]:
path = "input_sentences/grammatical_sentences.txt"
grammatical_test_sentences = load_sentences(path)

In [ ]:
print(grammatical_test_sentences)

In [ ]:
def tokenizer(sentences, word_to_idx):
    """
    Input: loaded text file
    Output: tokenized text
    """
    sentences = sentences.replace(".", ". <eos>")  # adding end of sentence symbols for tokenization

    sentences = sentences.split("<eos>")  # separates sentences
    sentences.pop()  # removes last element (a whitespace)
    print("number of test sentences: ", len(sentences))

    for i in range(len(sentences)):  # tokenizes sentences
        sentences[i] = sentences[i].replace(",", " ,").replace(".", " .").replace(":", " :").replace("?", " ?").replace("!", " !").replace(";", " ;")
        sentences[i] = sentences[i].lower()
        sentences[i] = sentences[i].split()

    tokenized_sentences = [[] for _ in range(len(sentences))]
    for i in range(len(sentences)):
        for j in range(len(sentences[i])):
            word = sentences[i][j]
            tokenized_sentences[i].append(word_to_idx[word] if word in word_to_idx else 2)

    return tokenized_sentences

In [ ]:
def generate_vocab_mappings(path):
    """
    Args:
        path: path to the vocabulary

    Returns:
        stoi: Word to index mapping
        itos: Index to word mapping
    """
    with open(path, "r") as inFile:
        itos = [x.split("\t")[0] for x in inFile.read().strip().split("\n")[:50003]]
    stoi = dict((tok, i) for i, tok in enumerate(itos))
    return itos, stoi

In [ ]:
CHAR_VOCAB_PATH = "vocabularies/german-wiki-word-vocab-50000.txt"
itos, stoi = generate_vocab_mappings(CHAR_VOCAB_PATH)

In [ ]:
tokens = tokenizer(grammatical_test_sentences, stoi)

In [ ]:
print(tokens)

### Quick & dirty sentences analyses

In [ ]:
df_gram = pd.read_csv("/Users/charlotterochereau/Documents/Stage_ENS/WIP/First_analyses/Gram_sentences.csv")
df_ungram = pd.read_csv("/Users/charlotterochereau/Documents/Stage_ENS/WIP/First_analyses/Ungram_sentences.csv")

In [ ]:
ungram_1_1 = [1.8197510245174993e-26, 1.1914022019967024e-27, 2.905147398911371e-25, 2.3633427269731422e-26, 3.7704084469221295e-27, 5.2638642072318534e-27, 1.402292291939187e-24, 1.7238227302671648e-26, 1.6281579625583875e-26, 1.1092214286634447e-25, 9.59501251410588e-27, 4.4234141588449915e-28, 1.402292291939187e-24, 1.7238227302671648e-26, 1.6281579625583875e-26, 1.1092214286634447e-25, 9.59501251410588e-27, 4.4234141588449915e-28]
ungram_1_2 = [4.1214189862240224e-23, 3.891355872215434e-25, 7.931403630787762e-25, 1.9560874427664567e-25, 6.49504147666468e-26, 3.0903963726161845e-26, 2.6540056944124747e-25, 2.741810523297171e-26, 2.9073107224273785e-24, 5.770835272924917e-25, 4.746384559065053e-26, 2.8796007097504793e-26, 2.650631326370158e-26, 3.632351977193616e-27, 3.146926389922294e-29, 1.5995362544401506e-28, 3.1307186838297904e-28, 1.5770543495382042e-30]
ungram_1_3 = [1.402292291939187e-24, 1.7238227302671648e-26, 1.6281579625583875e-26, 1.1092214286634447e-25, 9.59501251410588e-27, 4.4234141588449915e-28, 1.8197510245174993e-26, 1.1914022019967024e-27, 2.905147398911371e-25, 2.3633427269731422e-26, 3.7704084469221295e-27, 5.2638642072318534e-27, 1.8197510245174993e-26, 1.1914022019967024e-27, 2.905147398911371e-25, 2.3633427269731422e-26, 3.7704084469221295e-27, 5.2638642072318534e-27]
ungram_1_4 = [4.848404623066212e-24, 6.6826386836903885e-25, 3.1535708245714803e-25, 1.3065385940097741e-25, 3.1523635526991785e-25, 9.612121221685916e-26, 7.277993465923655e-29, 2.4098139123120457e-29, 3.7087371024664863e-28, 1.4132088852505134e-28, 2.0120026342628044e-29, 5.161271138810523e-30, 3.4190365313605426e-26, 1.5469232761252324e-26, 1.6420242448827547e-28, 2.1598599885827063e-29, 1.6734245649984437e-27, 1.0496114108377351e-28]
ungram_1_5 = [2.5532964160996946e-25, 3.773863859783631e-26, 8.2102592812064685e-25, 1.0090276094808538e-25, 1.6133005266061805e-26, 3.523486797396396e-25, 2.6540056944124747e-25, 2.741810523297171e-26, 2.9073107224273785e-24, 5.770835272924917e-25, 4.746384559065053e-26, 2.8796007097504793e-26, 3.137789196089694e-29, 6.987313047322733e-30, 1.4878892544205329e-27, 4.662804634422346e-28, 7.521223317666711e-30, 3.995289837694741e-29]
ungram_1_6 = [3.6469839691625454e-29, 3.8354232449422276e-29, 7.478270441275338e-29, 3.1733931035180563e-29, 1.0193431219013463e-28, 2.8684383119189244e-28, 4.848404623066212e-24, 6.6826386836903885e-25, 3.1535708245714803e-25, 1.3065385940097741e-25, 3.1523635526991785e-25, 9.612121221685916e-26, 7.277993465923655e-29, 2.4098139123120457e-29, 3.7087371024664863e-28, 1.4132088852505134e-28, 2.0120026342628044e-29, 5.161271138810523e-30]

nom_acc_prob = sum(ungram_1_1)/18
nom_dat_prob = sum(ungram_1_2)/18
acc_nom_prob = sum(ungram_1_3)/18
acc_dat_prob = sum(ungram_1_5)/18
dat_nom_prob = sum(ungram_1_4)/18
dat_acc_prob = sum(ungram_1_6)/18


print(nom_acc_prob,nom_dat_prob,acc_nom_prob,acc_dat_prob,dat_nom_prob,dat_acc_prob)




In [ ]:
gram_1 = [2.650631326370158e-26, 3.632351977193616e-27, 3.146926389922294e-29, 1.5995362544401506e-28, 3.1307186838297904e-28, 1.5770543495382042e-30, 4.1214189862240224e-23, 3.891355872215434e-25, 7.931403630787762e-25, 1.9560874427664567e-25, 6.49504147666468e-26, 3.0903963726161845e-26, 3.137789196089694e-29, 6.987313047322733e-30, 1.4878892544205329e-27, 4.662804634422346e-28, 7.521223317666711e-30, 3.995289837694741e-29, 2.5532964160996946e-25, 3.773863859783631e-26, 8.2102592812064685e-25, 1.0090276094808538e-25, 1.6133005266061805e-26, 3.523486797396396e-25, 2.6540056944124747e-25, 2.741810523297171e-26, 2.9073107224273785e-24, 5.770835272924917e-25, 4.746384559065053e-26, 2.8796007097504793e-26, 2.6540056944124747e-25, 2.741810523297171e-26, 2.9073107224273785e-24, 5.770835272924917e-25, 4.746384559065053e-26, 2.8796007097504793e-26]
gram_sum = sum(gram_1)
gram_prob = gram_sum/36
print(gram_prob)

In [ ]:
ungram_2_1 = [5.31314601588828e-30, 6.886424492228892e-32, 8.475041232806617e-29, 5.495574780508546e-32, 1.2533382031123979e-29, 8.81518222105264e-31, 3.398278826976099e-27, 6.206782093025597e-30, 2.4104654566804103e-30, 9.17160754877153e-31, 1.9311144308118907e-30, 1.815505518927516e-32, 3.398278826976099e-27, 6.206782093025597e-30, 2.4104654566804103e-30, 9.17160754877153e-31, 1.9311144308118907e-30, 1.815505518927516e-32]
ungram_2_2 = [2.1231143281667416e-26, 2.121711497643747e-28, 3.2257269643178144e-29, 2.654216621743849e-30, 2.6763518741843423e-30, 2.1209111817875933e-31, 6.275548479913818e-29, 3.9879446789649375e-30, 1.117288482254383e-28, 1.5568277862197476e-31, 1.416887561703195e-29, 2.31635958558783e-30, 3.479915017462916e-29, 5.6233052481009684e-31, 1.1058364494839749e-34, 7.666226938175838e-34, 6.894825821401168e-33, 8.031351957695822e-35]
ungram_2_3 = [3.398278826976099e-27, 6.206782093025597e-30, 2.4104654566804103e-30, 9.17160754877153e-31, 1.9311144308118907e-30, 1.815505518927516e-32, 5.31314601588828e-30, 6.886424492228892e-32, 8.475041232806617e-29, 5.495574780508546e-32, 1.2533382031123979e-29, 8.81518222105264e-31, 5.31314601588828e-30, 6.886424492228892e-32, 8.475041232806617e-29, 5.495574780508546e-32, 1.2533382031123979e-29, 8.81518222105264e-31]
ungram_2_4 = [4.567808230318964e-28, 6.916737865050997e-29, 1.1941629384747627e-28, 3.5724920906764817e-31, 2.3623994484612477e-29, 7.42744396310178e-30, 2.9213997495136082e-33, 1.045295375014449e-33, 1.9605233894712983e-33, 9.989604573420843e-34, 2.1280418206690624e-32, 2.946396782199197e-33, 1.8429925144250104e-29, 4.396032346227073e-30, 5.903801616320005e-34, 3.975066453320965e-34, 4.35777616605649e-33, 5.152681062074958e-34]
ungram_2_5 = [5.4681573885871936e-30, 1.613401868621344e-30, 1.4290787032995227e-28, 5.8022009768962934e-31, 1.0497495226291593e-29, 1.5857662086662557e-29, 6.275548479913818e-29, 3.9879446789649375e-30, 1.117288482254383e-28, 1.5568277862197476e-31, 1.416887561703195e-29, 2.31635958558783e-30, 5.973513392884645e-34, 5.0822216610912034e-34, 2.070371004562727e-32, 9.631641301395232e-33, 2.1046780076976132e-32, 3.076891586835523e-33]
ungram_2_6 = [2.676059387294212e-33, 1.9097021402139976e-33, 3.067201604717298e-32, 7.670651665388236e-33, 1.1897488385104387e-32, 2.4926400413017306e-32, 4.567808230318964e-28, 6.916737865050997e-29, 1.1941629384747627e-28, 3.5724920906764817e-31, 2.3623994484612477e-29, 7.42744396310178e-30, 2.9213997495136082e-33, 1.045295375014449e-33, 1.9605233894712983e-33, 9.989604573420843e-34, 2.1280418206690624e-32, 2.946396782199197e-33]

nom_acc_prob_2 = sum(ungram_2_1)/18
nom_dat_prob_2 = sum(ungram_2_2)/18
acc_nom_prob_2 = sum(ungram_2_3)/18
acc_dat_prob_2 = sum(ungram_2_5)/18
dat_nom_prob_2 = sum(ungram_2_4)/18
dat_acc_prob_2 = sum(ungram_2_6)/18


print(nom_acc_prob_2,nom_dat_prob_2,acc_nom_prob_2,acc_dat_prob_2,dat_nom_prob_2,dat_acc_prob_2)


In [ ]:
gram_2 = [3.479915017462916e-29, 5.6233052481009684e-31, 1.1058364494839749e-34, 7.666226938175838e-34, 6.894825821401168e-33, 8.031351957695822e-35, 2.1231143281667416e-26, 2.121711497643747e-28, 3.2257269643178144e-29, 2.654216621743849e-30, 2.6763518741843423e-30, 2.1209111817875933e-31, 5.973513392884645e-34, 5.0822216610912034e-34, 2.070371004562727e-32, 9.631641301395232e-33, 2.1046780076976132e-32, 3.076891586835523e-33, 5.4681573885871936e-30, 1.613401868621344e-30, 1.4290787032995227e-28, 5.8022009768962934e-31, 1.0497495226291593e-29, 1.5857662086662557e-29, 6.275548479913818e-29, 3.9879446789649375e-30, 1.117288482254383e-28, 1.5568277862197476e-31, 1.416887561703195e-29, 2.31635958558783e-30, 6.275548479913818e-29, 3.9879446789649375e-30, 1.117288482254383e-28, 1.5568277862197476e-31, 1.416887561703195e-29, 2.31635958558783e-30]
gram_sum_2 = sum(gram_2)
gram_prob_2 = gram_sum_2/36
print(gram_prob_2)